# Chapter 1

In [1]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx._
import scala.sys.process._

In [2]:
"head -n3 data/people.csv" !

1,Alice,20
2,Bob,18
3,Charlie,30


0

In [3]:
"head -n3 data/links.csv" !

1,2,friend
1,3,sister
3,2,boss


0

In [4]:
val people = sc.textFile("file:///home/sergey/Spark_GraphX/data/people.csv")
val links = sc.textFile("file:///home/sergey/Spark_GraphX/data/links.csv")

people = file:///home/sergey/Spark_GraphX/data/people.csv MapPartitionsRDD[1] at textFile at <console>:36
links = file:///home/sergey/Spark_GraphX/data/links.csv MapPartitionsRDD[3] at textFile at <console>:37


file:///home/sergey/Spark_GraphX/data/links.csv MapPartitionsRDD[3] at textFile at <console>:37

In [5]:
case class Person(name: String, age: Int)
type Connexion = String

defined class Person
defined type alias Connexion


In [6]:
val peopleRDD: RDD[(VertexId, Person)] = people.map{ line => 
    val row = line.split(",")
    (row(0).toInt, Person(row(1), row(2).toInt))
}

val linksRDD: RDD[Edge[Connexion]] = links.map{line => 
    val row = line.split(",")
    Edge(row(0).toInt, row(1).toInt, row(2))
}

val tinySocial: Graph[Person, Connexion] = Graph(peopleRDD, linksRDD)

tinySocial.cache()

peopleRDD = MapPartitionsRDD[4] at map at <console>:40
linksRDD = MapPartitionsRDD[5] at map at <console>:45
tinySocial = org.apache.spark.graphx.impl.GraphImpl@4c0b2433


org.apache.spark.graphx.impl.GraphImpl@4c0b2433

In [7]:
tinySocial.vertices.collect()

Array((4,Person(Dave,25)), (6,Person(Faith,21)), (8,Person(Harvey,47)), (2,Person(Bob,18)), (1,Person(Alice,20)), (3,Person(Charlie,30)), (7,Person(George,34)), (9,Person(Ivy,21)), (5,Person(Eve,30)))

In [8]:
tinySocial.edges.collect()

Array(Edge(1,2,friend), Edge(1,3,sister), Edge(2,4,brother), Edge(3,2,boss), Edge(4,5,client), Edge(1,9,friend), Edge(6,7,cousin), Edge(7,9,coworker), Edge(8,9,father))

In [9]:
val profLinks: List[Connexion] = List("coworker", "boss", "employee","client", "supplier")

profLinks = List(coworker, boss, employee, client, supplier)


List(coworker, boss, employee, client, supplier)

In [11]:
tinySocial.
    subgraph(t => profLinks.contains(t.attr)).
    triplets.
    collect.
    foreach(t => println(t.srcAttr.name + " is a " + t.attr + " of " + t.dstAttr.name))

Charlie is a boss of Bob
Dave is a client of Eve
George is a coworker of Ivy


# Chapter 2